# 02 – Data Cleaning & Quality Assurance 
### Smart City Energy Dataset (Asian Power Standards)

This notebook represents the **second step** of the Smart City Energy Analytics pipeline. 
While the previous notebook (`01_data_import_documented.ipynb`) focuses on loading the dataset, 
this notebook performs a **full data cleaning and quality assurance workflow** based on 
**Asian power grid standards**.

---

##  Notebook Goals

This notebook prepares the smart grid dataset for analytics and modeling by:

- Checking the **overall data quality and structure**
- Cleaning the **Country** column (removing non-country labels)
- Handling **missing values** in a robust way
- Removing **duplicate rows**
- Detecting **outliers** in:
- Voltage (V)
- Current (A)
- Power Consumption
- Removing outliers and validating the final cleaned dataset

The final cleaned dataset is stored in the variable **`df_final`** and is ready for:

- Exploratory Data Analysis (EDA)
- Statistical modeling
- Machine learning / forecasting



In [1]:
import os
from datetime import datetime
import pandas as pd
import numpy as np

print("=" * 80)
print("ASIAN POWER GRID – CLEANING PIPELINE STARTED")
print("=" * 80)

ASIAN POWER GRID – CLEANING PIPELINE STARTED


### Step 1 – Configuration
This step defines the input file path and the output folder for saving cleaned datasets. 
If the output folder does not already exist, it is automatically created. 
A timestamp is also generated to uniquely label exported files. 
This ensures that all cleaning operations have a consistent and organized file structure.

In [2]:
# =============================================================================
# 1. CONFIGURATION
# =============================================================================

INPUT_FILE = "C:/0_DA/Iot_DataAnalyst/smart_grid_dataset_city_modified.csv"
OUTPUT_FOLDER = "cleaned_datasets"

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
    print(f"[INFO] Created folder: {OUTPUT_FOLDER}")
else:
    print(f"[INFO] Output folder exists: {OUTPUT_FOLDER}")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")


[INFO] Created folder: cleaned_datasets


### Step 2 – Load Data
This step reads the input CSV file into a DataFrame and includes error handling to confirm successful loading or report issues, while also displaying the original dataset shape.

In [3]:
# =============================================================================
# 2. LOAD DATA
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 1] LOADING DATA")
print("=" * 80)

try:
    df = pd.read_csv(INPUT_FILE)
    print(f"[SUCCESS] Loaded file: {INPUT_FILE}")
    print(f"[INFO] Original shape: {df.shape}")
except FileNotFoundError:
    print(f"[ERROR] File not found: {INPUT_FILE}")
    raise
except Exception as e:
    print(f"[ERROR] Could not load file: {e}")
    raise

df_original = df.copy()


[STEP 1] LOADING DATA
[SUCCESS] Loaded file: C:/0_DA/Iot_DataAnalyst/smart_grid_dataset_city_modified.csv
[INFO] Original shape: (50010, 27)


### Step 3 – Data Quality Check
This step examines the dataset’s structure and completeness by displaying column information and counting missing values. 
It helps verify data types, detect potential issues, and ensure the dataset is ready for further cleaning steps.

In [4]:
# =============================================================================
# 3. DATA QUALITY CHECK
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 2] DATA QUALITY CHECK")
print("=" * 80)

df.info()
print("\n[INFO] Missing values per column:")
print(df.isnull().sum())



[STEP 2] DATA QUALITY CHECK
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50010 entries, 0 to 50009
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Timestamp                    50010 non-null  object 
 1   City                         50010 non-null  object 
 2   Name                         50010 non-null  object 
 3   Country                      50010 non-null  object 
 4   region                       50010 non-null  object 
 5   Voltage (V)                  50010 non-null  float64
 6   Current (A)                  50010 non-null  float64
 7   Power Consumption (kW)       50010 non-null  float64
 8   Reactive Power (kVAR)        50010 non-null  float64
 9   Power Factor                 50010 non-null  float64
 10  Solar Power (kW)             50010 non-null  float64
 11  Wind Power (kW)              50010 non-null  float64
 12  Grid Supply (kW)             50010 non-null  

### Step 4 – Handle Missing Numeric Values
In this step, all numeric columns are checked for missing values, and any gaps are filled using the column median. 
Median imputation is used because it is robust to outliers and preserves the central tendency of each feature. 
A cleaned copy of the dataset is created, and the process ensures no numeric missing values remain

In [6]:
# =============================================================================
# 4. HANDLE MISSING NUMERIC VALUES
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 3] HANDLING MISSING VALUES")
print("=" * 80)

df_cleaned = df.copy()
numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    missing_count = df_cleaned[col].isnull().sum()
    if missing_count > 0:
        median_val = df_cleaned[col].median()
        df_cleaned[col].fillna(median_val, inplace=True)
        print(f"[FIX] {col}: Filled {missing_count} missing values with median {median_val}")

print(f"[DEBUG] Remaining missing values: {df_cleaned.isnull().sum().sum()}")


[STEP 3] HANDLING MISSING VALUES
[DEBUG] Remaining missing values: 0


### Step 5 – Remove Duplicates
This step identifies any duplicate rows in the dataset and removes them to prevent repeated observations from biasing the analysis. 
Duplicate detection ensures data integrity, and only unique records are retained in the cleaned dataset.

In [7]:
# =============================================================================
# 5. REMOVE DUPLICATES
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 4] DUPLICATE REMOVAL")
print("=" * 80)

duplicate_count = df_cleaned.duplicated().sum()
print(f"[INFO] Duplicate rows found: {duplicate_count}")

if duplicate_count > 0:
    df_cleaned = df_cleaned.drop_duplicates()
    print(f"[SUCCESS] Removed {duplicate_count} duplicates")


[STEP 4] DUPLICATE REMOVAL
[INFO] Duplicate rows found: 10
[SUCCESS] Removed 10 duplicates


### Step 6 – Outlier Detection
In this step, outliers are identified using both domain knowledge and statistical rules. 
Voltage outliers are detected based on a realistic operating range for Asian grids (90–250 V), while Current and Power Consumption outliers are found using the IQR method. 
All detected outlier indices are collected to understand how many records may require removal or special handling in later cleaning steps.

In [8]:
# =============================================================================
# 6. OUTLIER DETECTION
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 5] OUTLIER DETECTION")
print("=" * 80)

ASIA_VOLTAGE_RANGE = {"min": 90, "max": 250}
outlier_indices = set()

# Voltage outliers (domain-based)
if "Voltage (V)" in df_cleaned.columns:
    v_outliers = df_cleaned[(df_cleaned["Voltage (V)"] < ASIA_VOLTAGE_RANGE["min"]) |
                            (df_cleaned["Voltage (V)"] > ASIA_VOLTAGE_RANGE["max"])]
    outlier_indices.update(v_outliers.index)
    print(f"[OUTLIER] Voltage: {len(v_outliers)}")

# IQR function
def detect_outliers_iqr(data, col):
    Q1, Q3 = data[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[col] < lower) | (data[col] > upper)]

# Current outliers
if "Current (A)" in df_cleaned.columns:
    c_outliers = detect_outliers_iqr(df_cleaned, "Current (A)")
    outlier_indices.update(c_outliers.index)
    print(f"[OUTLIER] Current (A): {len(c_outliers)}")

# Power consumption outliers
if "Power Consumption" in df_cleaned.columns:
    p_outliers = detect_outliers_iqr(df_cleaned, "Power Consumption")
    outlier_indices.update(p_outliers.index)
    print(f"[OUTLIER] Power Consumption: {len(p_outliers)}")

print(f"[SUMMARY] TOTAL OUTLIER ROWS: {len(outlier_indices)}")



[STEP 5] OUTLIER DETECTION
[OUTLIER] Voltage: 16
[OUTLIER] Current (A): 12
[SUMMARY] TOTAL OUTLIER ROWS: 16


### Step 7 – Remove Outliers
In this step, all rows identified as outliers in the previous stage are removed from the dataset. 
This ensures that extreme or unrealistic values do not distort analysis or modeling. 
The number of removed rows is reported to track data reduction.

In [9]:
# =============================================================================
# 7. REMOVE OUTLIERS
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 6] REMOVING OUTLIERS")
print("=" * 80)

before = len(df_cleaned)
df_final = df_cleaned.drop(index=outlier_indices)
after = len(df_final)

print(f"[INFO] Removed {before - after} rows")


[STEP 6] REMOVING OUTLIERS
[INFO] Removed 16 rows


### Step 8 – Clean DecommissionStatus
This step standardizes the mixed values in the `DecommissionStatus` column by extracting two clean fields: 
a categorical `OperationalStatus` indicating whether the sensor is active or decommissioned, and a `DecommissionDate` parsed from valid date entries. 
The original mixed-format column is then removed to improve data consistency and usability.

In [10]:
# =============================================================================
# 8. CLEAN DecommissionStatus (Mixed Types)
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 7] CLEANING 'DecommissionStatus'")
print("=" * 80)

if "DecommissionStatus" in df_final.columns:

    # Create new categorical status
    df_final["OperationalStatus"] = df_final["DecommissionStatus"].apply(
        lambda x: "Operational" if x == "Operational" else "Decommissioned"
    )

    # Extract real dates
    df_final["DecommissionDate"] = pd.to_datetime(
        df_final["DecommissionStatus"], errors="coerce"
    )

    # Drop original column
    df_final.drop(columns=["DecommissionStatus"], inplace=True)

    print("[SUCCESS] Cleaned DecommissionStatus")

else:
    print("[WARN] Column 'DecommissionStatus' missing; skipping cleanup")


[STEP 7] CLEANING 'DecommissionStatus'
[SUCCESS] Cleaned DecommissionStatus


### Step 9 – Date Cleaning
In this step, all date-related columns are converted into proper `datetime` format to ensure consistent handling of timestamps. 
Invalid or non-date values are safely converted to `NaT` using `errors='coerce'`, enabling accurate time-based analysis in later stages.

In [11]:
# =============================================================================
# 9. DATE CLEANING
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 8] DATE CLEANING")
print("=" * 80)

date_columns = ["Timestamp", "InstallationDate", "DecommissionDate"]

for col in date_columns:
    if col in df_final.columns:
        df_final[col] = pd.to_datetime(df_final[col], errors='coerce')
        print(f"[INFO] Converted {col} to datetime")



[STEP 8] DATE CLEANING
[INFO] Converted Timestamp to datetime
[INFO] Converted InstallationDate to datetime
[INFO] Converted DecommissionDate to datetime


### Step 10 – Drop Non-Informative Columns
This step removes columns that do not contribute meaningful information to the analysis. 
Since the `region` column contains only a single category, it provides no variability and is dropped to simplify the dataset.

In [ ]:
# =============================================================================
# 10. DROP USELESS COLUMNS (region)
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 9] DROPPING NON-INFORMATIVE COLUMNS")
print("=" * 80)

if "region" in df_final.columns:
    if df_final["region"].nunique() == 1:
        df_final.drop(columns=["region"], inplace=True)
        print("[INFO] Dropped column 'region' (only one category)")

### Step 11 – Categorical Encoding
In this step, categorical features are transformed into numeric format to prepare the dataset for modeling. 
Selected columns (`Name`, `City`, `Manufacturer`, `Sensor_ID`) are one-hot encoded, while `OperationalStatus` is converted into a binary indicator. 
This ensures all categorical information is represented in a machine-readable form

In [12]:
# =============================================================================
# 11. CATEGORICAL ENCODING
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 10] CATEGORICAL ENCODING")
print("=" * 80)

categorical_cols = ["Name", "City", "Manufacturer", "Sensor_ID"]

for col in categorical_cols:
    if col in df_final.columns:
        df_final = pd.get_dummies(df_final, columns=[col], drop_first=True)
        print(f"[ENCODE] One-hot encoded {col}")

# Binary encoding for OperationalStatus
if "OperationalStatus" in df_final.columns:
    df_final["OperationalStatus"] = df_final["OperationalStatus"].map({
        "Operational": 1,
        "Decommissioned": 0
    })
    print("[ENCODE] Encoded OperationalStatus (Operational=1, Decommissioned=0)")


[STEP 10] CATEGORICAL ENCODING
[ENCODE] One-hot encoded Name
[ENCODE] One-hot encoded City
[ENCODE] One-hot encoded Manufacturer
[ENCODE] One-hot encoded Sensor_ID
[ENCODE] Encoded OperationalStatus (Operational=1, Decommissioned=0)


### Step 12 – Save Cleaned Data and Report
In this final step, the fully cleaned dataset is exported as a new CSV file, and a detailed cleaning report is generated summarizing all transformations. 
The report includes row counts, removed duplicates and outliers, processed columns, and file locations, ensuring full transparency and reproducibility of the cleaning workflow.

In [13]:
# =============================================================================
# 12. SAVE CLEANED DATA + REPORT
# =============================================================================
print("\n" + "=" * 80)
print("[STEP 11] SAVING RESULTS")
print("=" * 80)

cleaned_file = os.path.join(OUTPUT_FOLDER, f"cleaned_data_{timestamp}.csv")
report_file = os.path.join(OUTPUT_FOLDER, f"cleaning_report_{timestamp}.txt")

df_final.to_csv(cleaned_file, index=False)
print(f"[SUCCESS] Cleaned data saved: {cleaned_file}")

report_text = f"""
CLEANING REPORT – ASIAN POWER GRID
===============================================
Timestamp: {timestamp}

Original rows:     {len(df_original)}
Final rows:        {len(df_final)}
Duplicates removed: {duplicate_count}
Outliers removed:   {before - after}

Processed columns:
- DecommissionStatus cleaned into OperationalStatus + DecommissionDate
- Dates converted to datetime
- region column dropped
- One-hot encoding for categorical columns
- Numeric missing values filled

Saved files:
- Cleaned CSV: {cleaned_file}
- Report:      {report_file}
"""

with open(report_file, "w", encoding="utf-8") as f:
    f.write(report_text)

print(f"[SUCCESS] Report saved: {report_file}")


[STEP 11] SAVING RESULTS
[SUCCESS] Cleaned data saved: cleaned_datasets\cleaned_data_20251205_130529.csv
[SUCCESS] Report saved: cleaned_datasets\cleaning_report_20251205_130529.txt


In [14]:
# =============================================================================
# DONE
# =============================================================================
print("\n" + "=" * 80)
print("CLEANING PIPELINE COMPLETED SUCCESSFULLY")
print("=" * 80)


CLEANING PIPELINE COMPLETED SUCCESSFULLY
